# IMPORTING PACKAGES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from torch.utils.data import DataLoader, Dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import * 
from torchvision import transforms
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.initializers import lecun_normal, HeNormal
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten
from sklearn.metrics import classification_report, roc_auc_score

# READING THE DATA AND GENERATING THE TRAIN AND TEST DATASET

In [ ]:
data = pd.read_csv("/kaggle/input/intras/data_only_numerical.csv")

,Unnamed: 0,target,age_approx,sex,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,iddx_full,iddx_1,tbp_lv_dnn_lesion_confidence
0,0,0,60.0,0,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,...,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,0,0,97.517282
1,1,0,60.0,0,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,...,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,0,0,3.141455
2,2,0,60.0,0,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,...,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,0,0,99.804040
3,3,0,65.0,0,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,...,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,0,0,99.989998
4,4,0,55.0,0,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,...,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,0,0,70.442510


In [ ]:
data = data.drop(['iddx_full','iddx_1','tbp_lv_dnn_lesion_confidence','tbp_lv_nevi_confidence', 'Unnamed: 0'],axis=1)

In [7]:
data_1 = pd.read_csv("/kaggle/input/intras/subject_data.csv")

/tmp/ipykernel_36/3951759125.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv("/kaggle/input/intras/subject_data.csv")


In [8]:
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import io
import numpy as np

file_path = "/kaggle/input/intras/images.hdf5"

# Filter data to get ISIC IDs where the target is 1
isic_codes = data_1[data_1["target"] >= 0]["isic_id"].tolist()
images = []
target = data_1['target'].values
with h5py.File(file_path, "r") as hdf:
    ls = list(hdf.keys())  # list all HDF5 keys
    for i in isic_codes:
        image_data = hdf[i]
        # Check if the dataset is scalar and contains bytes
        if image_data.shape == () and image_data.dtype.kind == 'S':
            # Read bytes and open as PIL image
            image_bytes = image_data[()]
            img_pil = Image.open(io.BytesIO(image_bytes))
            images.append(img_pil)
        else:
            print(f"Item '{i}' is not a scalar byte string")

In [11]:
data['images_idx'] = [i for i in range(0, len(target))]

In [12]:
mask = data['target'] == 1
df_ones = data[mask].copy()
df_zeros = data[~mask].copy()

In [13]:
from sklearn.model_selection import train_test_split
X_train_ones, X_test_ones, y_train_ones, y_test_ones = train_test_split(df_ones.drop(columns=['target']), df_ones['target'], test_size=0.3, random_state=42)

In [15]:
sampled_x_train_zeros, sampled_x_zeros, sampled_y_train_zeros, samples_y_zeros = train_test_split(df_zeros.drop(columns=['target']), df_zeros['target'], test_size=0.01, random_state=42)

In [17]:
X_TR_ZEROS, X_TEST_ZEROS, Y_TR_ZEROS, Y_TEST_ZEROS = train_test_split(sampled_x_zeros, samples_y_zeros, test_size=0.2, random_state=42)

In [19]:
# Features (X)
X_train = pd.concat([X_train_ones, X_TR_ZEROS], ignore_index=True)
X_test  = pd.concat([X_test_ones, X_TEST_ZEROS], ignore_index=True)

# Labels (y) - keep as DataFrame, not array
y_train = pd.concat([y_train_ones, Y_TR_ZEROS], ignore_index=True)
y_test  = pd.concat([y_test_ones, Y_TEST_ZEROS], ignore_index=True)

In [21]:
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [24]:
images_test = []
images_train = []
for i in X_test['images_idx'].to_numpy():
    images_test.append(images[i])
for i in X_train['images_idx'].to_numpy():
    images_train.append(images[i])

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(128),
    transforms.ToTensor()
])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pil_images, targets, X, transform=None):
        self.X = X
        self.images = pil_images
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        label = self.targets[idx]
        return image, self.X.iloc[idx].values, label

In [32]:
training = CustomDataset(images_train, y_train, X_train.drop(columns=['images_idx']), preprocess)
testing = CustomDataset(images_test, y_test, X_test.drop(columns=['images_idx']), preprocess)
train_dataloader = DataLoader(training, batch_size=128, shuffle=True, pin_memory=True)
test_dataloader = DataLoader(testing, batch_size=128, shuffle=True, pin_memory=True)

In [34]:
def dataloader_to_numpy(dataloader):
    images, X, y = [], [], []
    for imgs, x, labels in dataloader:
        imgs = imgs.permute(0, 2, 3, 1)  # (B, H, W, C)
        images.append(imgs.numpy())
        X.append(x.numpy())
        y.append(labels.numpy())
    return np.concatenate(images), np.concatenate(y), np.concatenate(X)

X_train_img, y_train_, X_train_ = dataloader_to_numpy(train_dataloader)
X_test_img, y_test_, X_test_ = dataloader_to_numpy(test_dataloader)

In [36]:
seed_value = 42
tf.random.set_seed(seed_value)   # TensorFlow seed
np.random.seed(seed_value)       # NumPy seed

# MODEL ONLY BASED ON IMAGE DATA

In [ ]:
# L2 regularization factor
reg_factor = 1e-3

model = Sequential([   
    Input(shape=(128, 128, 3, )),
    BatchNormalization(),
    
    Conv2D(64, (3, 3), activation='selu', kernel_initializer = lecun_normal(), kernel_regularizer=regularizers.l2(reg_factor)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.5),
    
    Conv2D(32, (3, 3), activation='selu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.45),
    
    Conv2D(16, (3, 3), activation='selu', kernel_initializer = lecun_normal(), kernel_regularizer=regularizers.l2(reg_factor)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Dropout(0.4),
    
    Flatten(),
    
    Dense(64, activation='selu', kernel_initializer = lecun_normal(), kernel_regularizer=regularizers.l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(32, activation='selu', kernel_initializer = lecun_normal(), kernel_regularizer=regularizers.l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(16, activation='selu', kernel_initializer = lecun_normal(), kernel_regularizer=regularizers.l2(reg_factor)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(1, activation='sigmoid')
])

# Compile
model.compile(optimizer=AdamW(learning_rate=2e-3),
              loss='binary_crossentropy',
              metrics=['recall', 'precision']
              )

model.summary()


In [ ]:
# Create the callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_model.h5",   # file name to save
    monitor="val_recall",         # quantity to monitor
    save_best_only=True,        # save only the best model
    mode="max",                 # "min" because lower val_loss is better
    verbose=1
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',    # Metric to monitor
    factor=0.4,            # Reduce LR by this factor (e.g., 0.01 → 0.001)
    patience=5,            # Wait this many epochs before reducing
    verbose=1,             # Print when LR changes
    min_lr=1e-6            # Lower bound for LR
)
# Fit model with callback
history = model.fit(
    X_train_img, y_train_,
    validation_data=(X_test_img, y_test_),
    epochs=100,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0:1, 1:5},
    verbose = 1
)

Epoch 1/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 1.4205 - precision: 0.2233 - recall: 0.6943
Epoch 1: val_recall improved from -inf to 0.02542, saving model to best_model.h5
59/59 ━━━━━━━━━━━━━━━━━━━━ 23s 159ms/step - loss: 1.4195 - precision: 0.2232 - recall: 0.6944 - val_loss: 0.8258 - val_precision: 1.0000 - val_recall: 0.0254 - learning_rate: 0.0020
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.2982 - precision: 0.2751 - recall: 0.6801
Epoch 2: val_recall did not improve from 0.02542
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 1.2971 - precision: 0.2754 - recall: 0.6806 - val_loss: 0.9156 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0020
Epoch 3/100
58/59 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.2111 - precision: 0.3570 - recall: 0.7034
Epoch 3: val_recall improved from 0.02542 to 0.17797, saving model to best_model.h5
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 1.2096 - precision: 0.3568 - recall: 0.7040 - val_loss:

KeyboardInterrupt: 

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/best_combined_model.h5')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/kaggle/working/best_combined_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [84]:
y_pred = model.predict(X_test_img)
for threshold in np.linspace(0, 0.5, 20):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(classification_report(y_test_, y_))

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step
threshold = 0.0
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       401
         1.0       0.23      1.00      0.37       118

    accuracy                           0.23       519
   macro avg       0.11      0.50      0.19       519
weighted avg       0.05      0.23      0.08       519

threshold = 0.02631578947368421
              precision    recall  f1-score   support

         0.0       0.95      0.40      0.56       401
         1.0       0.31      0.93      0.47       118

    accuracy                           0.52       519
   macro avg       0.63      0.67      0.52       519
weighted avg       0.81      0.52      0.54       519

threshold = 0.05263157894736842
              precision    recall  f1-score   support

         0.0       0.92      0.64      0.76       401
         1.0       0.40      0.82      0.54       118

    accuracy                           0.68       519
   macro

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# MODEL ONLY BASED ON TABULAR DATA

In [85]:
# MODEL FOR THE NUMERICAL FEATURES
model_num = Sequential([
    Input(shape=(34,)),
    
    BatchNormalization(),
    
    Dense(128, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(16, activation='elu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(1e-3)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(1, activation='sigmoid'),
])
model_num.compile(optimizer=AdamW(learning_rate=2e-3),
              loss='binary_crossentropy',
              metrics=['recall', 'precision']
              )
model_num.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_47          │ (None, 34)             │           136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │         4,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,761 (57.66 KB)

 Trainable params: 14,277 (55.77 KB)

 Non-trainable params: 484 (1.89 KB)

In [ ]:
# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model.h5",  # save best combined model
    monitor="val_recall",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.4,
    patience=5,
    verbose=1,
    min_lr=1e-6
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 5},  # handle imbalance
    verbose=1
)

NameError: name 'X_train_img' is not defined

In [87]:
y_pred = model_num.predict(X_test_)
for threshold in np.linspace(0, 0.5, 20):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(classification_report(y_test_, y_))

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
threshold = 0.0
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       401
         1.0       0.23      1.00      0.37       118

    accuracy                           0.23       519
   macro avg       0.11      0.50      0.19       519
weighted avg       0.05      0.23      0.08       519

threshold = 0.02631578947368421
              precision    recall  f1-score   support

         0.0       0.96      0.81      0.88       401
         1.0       0.58      0.89      0.70       118

    accuracy                           0.83       519
   macro avg       0.77      0.85      0.79       519
weighted avg       0.87      0.83      0.84       519

threshold = 0.05263157894736842
              precision    recall  f1-score   support

         0.0       0.96      0.85      0.90       401
         1.0       0.62      0.87      0.73       118

    accuracy                           0.85       519
   macro

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# HYBRID MODEL USING BOTH IMAGE AND TABULAR DATA

### CUSTOM METRIC

In [ ]:
class RecallWithThreshold(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.5, name='recall_with_threshold', **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred >= self.threshold, tf.float32)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.recall.result()

    def reset_state(self):
        self.recall.reset_state()


### CUSTOM LOSS

In [ ]:
def binary_focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)  # for numerical stability
        cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        focal_weight = tf.where(tf.equal(y_true, 1), (1 - y_pred) ** gamma, y_pred ** gamma)
        loss = alpha * focal_weight * cross_entropy
        return tf.reduce_mean(loss)
    return loss

### MODEL DIFINITION

In [ ]:
# L2 regularization factor
reg_factor = 1e-4
# -------- Image branch --------
image_input = Input(shape=(128, 128, 3), name='image_input')  # example shape
x1 = BatchNormalization()(image_input)

x1 = Conv2D(128, (3, 3), activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

x1 = Conv2D(96, (3, 3), activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Conv2D(64, (3, 3), activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)
x1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

x1 = Conv2D(15, (3, 3), activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = MaxPooling2D((2, 2))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Flatten()(x1)

x1 = Dense(128, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x1 = Dense(96, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x1 = Dense(64, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)
x1 = Dense(32, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x1 = Dense(16, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

# -------- Numerical branch --------
num_input = Input(shape=(34,), name='num_input')  # example shape
x2 = BatchNormalization()(num_input)

# x2 = Dense(48, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x2)
# x2 = BatchNormalization()(x2)
# x2 = Dropout(0.2)(x2)

# x2 = Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x2)
# x2 = BatchNormalization()(x2)
# x2 = Dropout(0.2)(x2)

# x2 = Dense(16, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(x2)

# -------- Merge branches --------
merged = Concatenate()([x1, x2])
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(64, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.2)(merged)
merged = Dense(16, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.1)(merged)
merged = Dense(64, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.1)(merged)
merged = Dense(16, activation='relu', kernel_initializer = HeNormal(), kernel_regularizer=regularizers.l2(reg_factor))(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.1)(merged)
output = Dense(1, activation='sigmoid')(merged)  # binary classification example

# -------- Build model --------
model = Model(inputs=[image_input, num_input], outputs=output)

# Compile
model.compile(optimizer=AdamW(learning_rate=1e-3), loss=binary_focal_loss(), metrics=['precision', RecallWithThreshold(0.3), F1ScoreWithThreshold()])
model.summary()

Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         12 │ image_input[0][0] │
│ (BatchNormalizatio… │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 126, 126,  │      3,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_59    │ (None, 63, 63,    │          0 │ conv2d_45[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 63, 63,    │        512 │ max_pooling2d_59… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_212         │ (None, 63, 63,    │          0 │ batch_normalizat… │
│ (Dropout)           │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 61, 61,    │    110,688 │ dropout_212[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_60    │ (None, 30, 30,    │          0 │ conv2d_46[0][0]   │
│ (MaxPooling2D)      │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 30, 30,    │        384 │ max_pooling2d_60… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_213         │ (None, 30, 30,    │          0 │ batch_normalizat… │
│ (Dropout)           │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 28, 28,    │     55,360 │ dropout_213[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_61    │ (None, 14, 14,    │          0 │ conv2d_47[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ max_pooling2d_61… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_214         │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 12, 12,    │     18,464 │ dropout_214[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_62    │ (None, 6, 6, 32)  │          0 │ conv2d_48[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 6, 32)  │        128 │ max_pooling2d_62

 Total params: 231,560 (904.53 KB)

 Trainable params: 229,724 (897.36 KB)

 Non-trainable params: 1,836 (7.17 KB)

In [ ]:
# Create the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath="best_combined_model.h5",  # save best combined model
    monitor="val_f1_score_with_threshold",               # monitor recall
    save_best_only=True,
    mode="max",
    verbose=1
)

# Reduce LR on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_f1_score_with_threshold',
    factor=0.9,
    patience=5,
    verbose=1,
    min_lr=1e-5
)

# Fit combined model
history = model.fit(
    [X_train_img, X_train_],  # both image and numerical inputs
    y_train_,
    validation_data=([X_test_img, X_test_], y_test_),
    epochs=100,
    batch_size = 51,
    callbacks=[checkpoint_cb, reduce_lr],
    class_weight={0: 1, 1: 5},  # handle imbalance
    verbose=1
)

Epoch 1/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - f1_score_with_threshold: 0.1467 - loss: 0.3107 - precision: 0.0758 - recall_with_threshold: 0.7165
Epoch 1: val_f1_score_with_threshold improved from -inf to 0.21526, saving model to best_combined_model.h5
69/69 ━━━━━━━━━━━━━━━━━━━━ 59s 349ms/step - f1_score_with_threshold: 0.1467 - loss: 0.3102 - precision: 0.0759 - recall_with_threshold: 0.7170 - val_f1_score_with_threshold: 0.2153 - val_loss: 0.2883 - val_precision: 0.1016 - val_recall_with_threshold: 0.9322 - learning_rate: 0.0010
Epoch 2/100
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - f1_score_with_threshold: 0.1435 - loss: 0.2305 - precision: 0.1257 - recall_with_threshold: 0.6792
Epoch 2: val_f1_score_with_threshold improved from 0.21526 to 0.27451, saving model to best_combined_model.h5
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - f1_score_with_threshold: 0.1437 - loss: 0.2304 - precision: 0.1256 - recall_with_threshold: 0.6799 - val_f1_score_with_threshold: 0.2745 - val_loss: 0

In [224]:
model = tf.keras.models.load_model(
    '/kaggle/working/best_combined_model.h5',
    custom_objects={'binary_focal_loss': binary_focal_loss()},
    compile = False
)

In [225]:
y_pred = model.predict([X_test_img, X_test_])
for threshold in np.linspace(0, 0.5, 50):
    y_ = y_pred > threshold
    print(f'threshold = {threshold}')
    print(f'roc_auc_score = {roc_auc_score(y_test_, y_)}')
    print(classification_report(y_test_, y_))

29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step
threshold = 0.0
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       802
         1.0       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.01020408163265306
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       802
         1.0       0.13      1.00      0.23       118

    accuracy                           0.13       920
   macro avg       0.06      0.50      0.11       920
weighted avg       0.02      0.13      0.03       920

threshold = 0.02040816326530612
roc_auc_score = 0.5
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       802
         1.0       0.13      1.00      0.23       118

  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

         0.0       0.96      0.93      0.94       802
         1.0       0.60      0.75      0.67       118

    accuracy                           0.90       920
   macro avg       0.78      0.84      0.81       920
weighted avg       0.92      0.90      0.91       920

threshold = 0.32653061224489793
roc_auc_score = 0.8211462868253095
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95       802
         1.0       0.63      0.70      0.66       118

    accuracy                           0.91       920
   macro avg       0.79      0.82      0.81       920
weighted avg       0.91      0.91      0.91       920

threshold = 0.336734693877551
roc_auc_score = 0.806067458472463
              precision    recall  f1-score   support

         0.0       0.95      0.94      0.95       802
         1.0       0.63      0.67      0.65       118

    accuracy                           0.91       9